In [5]:
!pip install gdal


  Using cached gdal-3.10.tar.gz (848 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build gdal


  error: subprocess-exited-with-error
  
  × Building wheel for gdal (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [148 lines of output]
      Using numpy 2.2.1
      C:\Users\Javier\AppData\Local\Temp\pip-build-env-96ppjf0u\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:79: SetuptoolsWarning: `extras_require` overwritten in `pyproject.toml` (optional-dependencies)
        corresp(dist, value, root_dir)
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdal.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdalconst.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdalnumeric.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gdal_array.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\gnm.py -> build\lib.win-amd64-cpython-311\osgeo
      copying osgeo\ogr.py -> build\lib.win-amd64-cpython-311\osgeo


In [ ]:
import json
import psycopg2
from psycopg2.extras import Json



# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='crop-health-db',
    user='master',
    password='JFKae$2341',
    host='crop-health-db.cv0iskeoocuw.eu-north-1.rds.amazonaws.com',
    port='5432'
)
cursor = conn.cursor()

# Load the JSON file
with open('C:\\Users\\Javier\\Desktop\\CROP\\fields_data.json', 'r') as file:
    data = json.load(file)


feature = data['features'][0]

client_id = feature['properties']['ID'][:4]  # First 4 characters for client_id
field_id = feature['properties']['ID'][4:6]  # Next 2 characters for field_id
crop_type = feature['properties']['CROP']    # Crop type value
geometry = feature['geometry']               # Polygon coordinates

# Extract the first coordinate of the polygon
first_coordinate = geometry['coordinates'][0][0]
lon, lat = first_coordinate[0], first_coordinate[1]

# Convert to MGRS
mgrs = MGRS()
mgrs_code = mgrs.forward(lat, lon)
print(mgrs_code)



ModuleNotFoundError: No module named 'geographiclib.mgrs'

In [ ]:
# Extract the Sentinel-2 path components
utm_zone = mgrs_code[:2]  # First 2 characters (e.g., '30')
lat_band = mgrs_code[2]   # Third character (e.g., 'T')
grid_square = mgrs_code[3:5]  # Fourth and fifth characters (e.g., 'TK')

# Construct Sentinel-2 query path
sentinel2_query = f"tiles/{utm_zone}/{lat_band}/{grid_square}/"

# Debugging: Print the values to verify correctness
print(f"Feature {feature['properties']['ID']} sentinel2_query: {sentinel2_query}")

try:
    # Insert the data into the fields table
    cursor.execute("""
        INSERT INTO fields (client_id, field_id, crop_type, polygon, sentinel2_query, created_at)
        VALUES (%s, %s, %s, ST_GeomFromGeoJSON(%s), %s, NOW());
    """, (client_id, field_id, crop_type, json.dumps(geometry), sentinel2_query))
    print(f"Inserted feature with ID {feature['properties']['ID']}")
except Exception as e:
    print(f"Error inserting feature with ID {feature['properties']['ID']}: {e}")

# Commit the transaction
conn.commit()

# Verify insertion
cursor.execute("SELECT * FROM fields LIMIT 10;")
rows = cursor.fetchall()
print("First 10 rows in the fields table:")
for row in rows:
    print(row)

# Close the connection
cursor.close()
conn.close()

In [30]:
import boto3
import rasterio
import numpy as np
from datetime import datetime, timedelta
from rasterio.io import MemoryFile

# Initialize S3 client
s3_client = boto3.client("s3")
BUCKET_NAME = "sentinel-s2-l2a"

# Compute date string
metrics_date = datetime.today() - timedelta(days=4)
metrics_date_str = f"{metrics_date.year}/{metrics_date.month}/{metrics_date.day}"

# Define the file path
sentinel2_query = "tiles/30/T/TK/"
FILE_PATH = f"{sentinel2_query}{metrics_date_str}/0/R10m/B08.jp2"

# Load file from S3
response = s3_client.get_object(Bucket=BUCKET_NAME, Key=FILE_PATH)

# Read the entire file into memory (important for large files)
file_bytes = response["Body"].read()

# Read file as a NumPy array using rasterio
with MemoryFile(file_bytes) as memfile:
    with memfile.open() as dataset:
        img_array = dataset.read()  # Reads all bands as a NumPy array
        profile = dataset.profile  # Metadata

# Print array shape and dtype
print(f"Array shape: {img_array.shape}")  # (bands, height, width)
print(f"Data type: {img_array.dtype}")
print(f"Metadata: {profile}")


In [34]:
path = "C:/Users/Javier/Downloads/TCI.jp2"

with rasterio.open(path) as data:
    raster_data = data.read(1)  # Read the first band into a NumPy array

In [ ]:
file = "tiles/30/T/TK/2025/1/26/0/R10m/TCI.jp2"
BUCKET_NAME = "sentinel-s2-l2a"
file_path = "C:/Users/Javier/Downloads/TCI.jp2"
"C:\Users\Javier\Downloads\IATA"

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [1]:
import boto3

# Initialize the S3 client with no credentials (for public access)
s3_client = boto3.client('s3', region_name='eu-central-1', config=boto3.session.Config(signature_version=None))

# Bucket name
bucket_name = 'ecmwf-forecasts'

# List objects in the bucket
response = s3_client.list_objects_v2(Bucket=bucket_name)

# Check if the bucket has any objects
if 'Contents' in response:
    print("Objects in the bucket:")
    for obj in response['Contents']:
        print(f" - {obj['Key']}")
else:
    print("No objects found in the bucket.")



Objects in the bucket:
 - 20230118/00z/0p4-beta/enfo/20230118000000-0h-enfo-ef.grib2
 - 20230118/00z/0p4-beta/enfo/20230118000000-0h-enfo-ef.index
 - 20230118/00z/0p4-beta/enfo/20230118000000-102h-enfo-ef.grib2
 - 20230118/00z/0p4-beta/enfo/20230118000000-102h-enfo-ef.index
 - 20230118/00z/0p4-beta/enfo/20230118000000-105h-enfo-ef.grib2
 - 20230118/00z/0p4-beta/enfo/20230118000000-105h-enfo-ef.index
 - 20230118/00z/0p4-beta/enfo/20230118000000-108h-enfo-ef.grib2
 - 20230118/00z/0p4-beta/enfo/20230118000000-108h-enfo-ef.index
 - 20230118/00z/0p4-beta/enfo/20230118000000-111h-enfo-ef.grib2
 - 20230118/00z/0p4-beta/enfo/20230118000000-111h-enfo-ef.index
 - 20230118/00z/0p4-beta/enfo/20230118000000-114h-enfo-ef.grib2
 - 20230118/00z/0p4-beta/enfo/20230118000000-114h-enfo-ef.index
 - 20230118/00z/0p4-beta/enfo/20230118000000-117h-enfo-ef.grib2
 - 20230118/00z/0p4-beta/enfo/20230118000000-117h-enfo-ef.index
 - 20230118/00z/0p4-beta/enfo/20230118000000-120h-enfo-ef.grib2
 - 20230118/00z/0p4-b